In [175]:
import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
import tensorflow as tf
import pickle5 as pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
from random import shuffle
import datetime

In [176]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [177]:
# import pathlib
# data_dir = pathlib.Path('E:/Users/sentinel_industry/downloaded_aois/')


In [178]:
x_size = 230
y_size = 230
def load_features(name):
    decoded = name.decode("UTF-8")
    if os.path.exists(decoded):
        with open(decoded, 'rb') as f:
            file = pickle.load(f)
            label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
            if label == 'coal':
                label = [1,0,0]
            elif label == 'steel':
                label = [0,1,0]
            else: label = [0,0,1]
            if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
                print("oh oh, downloaded patch too smol:", decoded)
            B02, B03, B04, B08 = file['B02'][0][0:x_size,0:y_size], file['B03'][0][0:x_size,0:y_size], file['B04'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
            for channel in [ B02, B03, B04, B08]: channel = channel/channel.max()
            features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
            # features = np.expand_dims(features, axis=0)
            return features, label
            # I have commented the line below but this should return
            # the features and the label in a one hot vector
            # return file['features'], file['targets']
    else:
        print("Something went wrong!")
        exit(-1)

In [179]:
path = 'E:/Users/sentinel_industry/downloaded_aois/coal/'

pkl_files = glob.glob((path+"*.pickle"))


In [180]:
print(len(pkl_files))

2258


In [181]:
path_steel = 'E:/Users/sentinel_industry/downloaded_aois/steel/'
pkl_files_steel = glob.glob((path_steel+"*.pickle"))

In [182]:
path_steel_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/steel/no_ind/'
pkl_files_steel_no_ind = glob.glob((path_steel_no_ind+"*.pickle"))

In [183]:
path_coal_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/coal/no_ind/'
pkl_files_coal_no_ind = glob.glob((path_coal_no_ind+"*.pickle"))

In [184]:
print(len(pkl_files_steel))

1722


In [185]:
pkl_files = pkl_files + pkl_files_steel + pkl_files_steel_no_ind + pkl_files_coal_no_ind
shuffle(pkl_files)
# pkl_files=pkl_files[0:100]

In [186]:
# from itertools import chain

# pkl_files = list(chain.from_iterable(zip(pkl_files, pkl_files_steel)))


In [187]:
print(len(pkl_files))

8077


In [188]:
dataset_len=len(pkl_files)

In [189]:
dataset = tf.data.Dataset.from_tensor_slices((pkl_files))

def data_loader(filename):
    features, labels = tf.numpy_function(load_features, [filename], [tf.uint16, tf.int32])
    # features.set_shape((None, 242,242,4))
    # labels.set_shape(( 1))
    return features, labels

dataset = dataset.map(data_loader)
               # load_features, [filename], [tf.uint16, tf.string])))

In [190]:
dataset.shuffle(100, seed=42)

train_size = np.floor(0.8 * dataset_len)
valid_size = np.floor(0.1 * dataset_len)
test_size = np.floor(0.1 * dataset_len)

train = dataset.take(train_size, )
remaining = dataset.skip(train_size)
valid = remaining.take(valid_size,
)
test = remaining.skip(valid_size)

In [191]:
train = train.shuffle(100,  reshuffle_each_iteration=True, )
valid = valid.shuffle(100,  reshuffle_each_iteration=True, )


In [194]:
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,4)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?

    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
])

In [195]:
# dataset = dataset.map((
#   lambda x, y: (resize_and_rescale(x), y)))

In [196]:
for image_batch, labels_batch in dataset:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(230, 230, 4)
(3,)


In [197]:
train

<ShuffleDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.uint16, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [198]:
pwd

'E:\\Users\\sentinel_industry'

In [199]:
# model = tf.keras.Sequential([
#     resize_rescale_augment,
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(3)
# ])
model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='relu'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=max ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/new/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.05)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [200]:
model.build()

In [201]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_13 (Sequential)  (None, 120, 120, 4)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 120, 120, 3)       303       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_9 (Flatten)         (None, 32768)             0         
                                                                 
 dense_11 (Dense)            (None, 3)                 98307     
                                                                 
Total params: 23,686,322
Trainable params: 23,633,202
Non-trainable params: 53,120
_________________________________________________________________


In [202]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [203]:
# !env LD_LIBRARY_PATH="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_LIBRARY_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_INCLUDE_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# # !set LD_INCLUDE_PATH=:/usr/local/cuda/include:/usr/local/cuda/extras/CUPTI/include

In [204]:
!env LD_LIBRARY_PATH

env: â€˜LD_LIBRARY_PATHâ€™: No such file or directory


In [205]:
%tensorboard --logdir='E:/Users/sentinel_industry/logs/train/new'

Reusing TensorBoard on port 6006 (pid 22096), started 2:33:05 ago. (Use '!kill 22096' to kill it.)

In [ ]:
epochs=50
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=1,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Epoch 1/50
161/161 [==============================] - 413s 3s/step - loss: 3.5494 - categorical_accuracy: 0.4899 - val_loss: 0.9829 - val_categorical_accuracy: 0.5750 - lr: 0.0010
Epoch 2/50
161/161 [==============================] - 349s 2s/step - loss: 1.7942 - categorical_accuracy: 0.5719 - val_loss: 1912.9668 - val_categorical_accuracy: 0.5750 - lr: 0.0010
Epoch 3/50
161/161 [==============================] - 356s 2s/step - loss: 1.7061 - categorical_accuracy: 0.6480 - val_loss: 0.9800 - val_categorical_accuracy: 0.5750 - lr: 0.0010
Epoch 4/50
161/161 [==============================] - 325s 2s/step - loss: 1.0988 - categorical_accuracy: 0.6882 - val_loss: 0.8159 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 5/50
161/161 [==============================] - 361s 2s/step - loss: 0.9553 - categorical_accuracy: 0.7023 - val_loss: 0.6962 - val_categorical_accuracy: 0.6750 - lr: 0.0010
Epoch 6/50
161/161 [==============================] - 343s 2s/step - loss: 1.0961 - categorical_a

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    next_element = iterator.get_next()
    im = resize_rescale_augment(next_element[0])
    plt.imshow(np.flip(im[0,:,:,0:3], axis=2))
    plt.title(convert_onehot(next_element))

In [ ]:
def convert_onehot(next_element):
    if (next_element[1].numpy() == np.array([0, 0, 1])).all(): return 'no industry'
    elif (next_element[1].numpy() == np.array([0, 1, 0])).all(): return 'steel'
    elif (next_element[1].numpy() == np.array([1, 0, 0])).all(): return 'coal'
    else: return 'not valid'

In [ ]:
next_element[1].numpy().all()

In [ ]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("module.name", "/path/to/file.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = foo
spec.loader.exec_module(foo)

In [ ]:
! PATH